<a href="https://colab.research.google.com/github/michaelwnau/aws-sagemaker-notebooks/blob/main/streamflow_infra_fpe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install diagrams

In [ ]:
# diagram.py
from diagrams import Diagram
from diagrams.aws.compute import EC2
from diagrams.aws.database import RDS
from diagrams.aws.network import ELB

with Diagram("Web Service", show=False):
    ELB("lb") >> EC2("web") >> RDS("userdb")

In [ ]:
from diagrams import Diagram, Cluster, Edge
from diagrams.aws.compute import Lambda
from diagrams.aws.ml import Sagemaker
from diagrams.aws.database import Dynamodb
from diagrams.aws.security import Cognito
from diagrams.aws.storage import S3
from diagrams.aws.integration import SNS
from diagrams.aws.ml import SagemakerModel
from diagrams.aws.management import Cloudformation

with Diagram("AWS Architecture", show=False, direction="TB") as diag:
    with Cluster("Storage"):
        deployment_bucket = S3("Deployment Bucket")
        storage_bucket = S3("Storage Bucket")
        website_bucket = S3("Website Bucket")
        models_bucket = S3("Models Bucket")

    with Cluster("Compute"):
        api_lambda = Lambda("API")
        trigger_lambda = Lambda("Trigger")
        worker_lambda = Lambda("Worker")

    with Cluster("Database"):
        db = Dynamodb("Database")

    with Cluster("Authentication"):
        auth = Cognito("Auth")

    with Cluster("Integration"):
        sns = SNS("Subscriptions")

    cloudformation = Cloudformation("CloudFormation")

    with Cluster("Machine Learning"):
        sagemaker_pipeline = Sagemaker("SageMaker Pipeline")
        resnet_model = SagemakerModel("Resnet-18")

    # Connections
    api_lambda >> cloudformation
    trigger_lambda >> cloudformation
    worker_lambda >> cloudformation

    deployment_bucket >> cloudformation
    storage_bucket >> cloudformation
    website_bucket >> cloudformation
    models_bucket >> cloudformation

    db >> cloudformation
    auth >> cloudformation
    sns >> cloudformation

    trigger_lambda >> sns

    # Sending batch images to SageMaker for inference
    models_bucket >> Edge(label="Batch Images") >> sagemaker_pipeline >> resnet_model

# Save the diagram
# diag.render("aws_architecture_diagram_extended.png")


In [ ]:
from diagrams import Diagram, Cluster
from diagrams.aws.compute import Lambda
from diagrams.aws.integration import SNS, SQS
from diagrams.aws.database import Dynamodb
from diagrams.aws.storage import S3
from diagrams.aws.ml import Sagemaker
from diagrams.aws.analytics import Kinesis
from diagrams.aws.management import Cloudwatch, Cloudtrail
from diagrams.aws.security import IAM

with Diagram("Automated ML Pipeline Architecture", show=False, direction="LR"):
    with Cluster("Data Ingestion"):
        kinesis = Kinesis("Kinesis")

    with Cluster("Data Processing"):
        lambda_processing = Lambda("Data Processing")

    with Cluster("Data Storage"):
        s3 = S3("S3")
        dynamodb = Dynamodb("DynamoDB")

    with Cluster("Model Training"):
        sagemaker_pipelines = Sagemaker("Sagemaker Pipelines")

    with Cluster("Performance Evaluation"):
        model_monitor = Sagemaker("Model Monitor")
        cloudwatch = Cloudwatch("CloudWatch")

    with Cluster("Prediction & Inference"):
        lambda_inference = Lambda("Batch Inference Trigger")
        sagemaker_endpoints = Sagemaker("Real-Time Inference")

    with Cluster("Monitoring & Security"):
        cloudtrail = Cloudtrail("CloudTrail")
        iam = IAM("IAM")

    with Cluster("Notification & Workflow"):
        sns = SNS("SNS")
        sqs = SQS("SQS")

    # Data flow
    kinesis >> lambda_processing >> s3
    sagemaker_pipelines >> model_monitor
    sagemaker_pipelines >> sagemaker_endpoints
    s3 >> dynamodb
    lambda_inference >> sagemaker_endpoints
    cloudwatch >> sns
    cloudtrail >> sqs

    # Service connections
    lambda_processing >> cloudwatch
    sagemaker_endpoints >> cloudwatch
    cloudwatch >> cloudtrail
    model_monitor >> sns
    sns >> iam
    sqs >> iam
